In [1]:
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.regression.LinearRegression
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.Row
import breeze.linalg.DenseMatrix
import breeze.linalg.DenseVector
import breeze.linalg.pinv

val spark = SparkSession.builder()
  .appName("LR")
  .getOrCreate()

Intitializing Scala interpreter ...

Spark Web UI available at http://10.111.0.197:4040
SparkContext available as 'sc' (version = 3.3.2, master = local[*], app id = local-1685553233042)
SparkSession available as 'spark'


23/05/31 20:14:05 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.regression.LinearRegression
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.Row
import breeze.linalg.DenseMatrix
import breeze.linalg.DenseVector
import breeze.linalg.pinv
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@17ec4e00


# Generate Data

In [2]:
val cefs = DenseVector(1.5, 0.3, -0.7)

val data = Seq.tabulate(131072) { _ =>
  val features = DenseVector.rand[Double](3)
  val label = cefs.dot(features) + breeze.stats.distributions.Gaussian(0, 0.1).draw()
  (label, Vectors.dense(features.toArray))
}

val df = spark.createDataFrame(data).toDF("label", "features")

coefficients: breeze.linalg.DenseVector[Double] = DenseVector(1.5, 0.3, -0.7)
data: Seq[(Double, org.apache.spark.ml.linalg.Vector)] = List((0.86665430003602,[0.6193465950662702,0.34453194900047523,0.2796787132033485]), (-0.43910030914423903,[0.0428064885515973,0.7484349824569834,0.5224699554091228]), (0.4177974879173749,[0.43025939511013167,0.033571346038936856,0.13891645114946138]), (-0.03425888691346103,[0.2476284683201302,0.8836627646653386,0.8576514055490709]), (0.12395607909528465,[0.17712783639609309,0.2150118935909,0.47950810314073666]), (1.0187711286561896,[0.9426804200392327,0.6761815189643852,0.8443157863012432]), (0.9355317012970494,[0.53472371088561,0.6818271330231482,0.15434489126312534]), (1.3361904598837961,[0.8266541028472159,0.3977865122070774,0.22823501387852763]), (0...


# LR

In [3]:
val Array(trainingData, testData) = df.randomSplit(Array(0.8, 0.2))

val LR = new LinearRegression().setMaxIter(10)

val LRModel = LR.fit(trainingData)
val preds = LRModel.transform(testData)

val eval = new org.apache.spark.ml.evaluation.RegressionEvaluator()
  .setLabelCol("label")
  .setPredictionCol("prediction")
  .setMetricName("mse")

val mse = eval.evaluate(preds)
println(s"$mse")

23/05/31 20:14:15 WARN TaskSetManager: Stage 0 contains a task of very large size (1940 KiB). The maximum recommended task size is 1000 KiB.
23/05/31 20:14:16 WARN Instrumentation: [975b337a] regParam is zero, which might cause numerical instability and overfitting.
23/05/31 20:14:17 WARN TaskSetManager: Stage 1 contains a task of very large size (1940 KiB). The maximum recommended task size is 1000 KiB.
23/05/31 20:14:20 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK
23/05/31 20:14:21 WARN TaskSetManager: Stage 3 contains a task of very large size (1940 KiB). The maximum recommended task size is 1000 KiB.
23/05/31 20:14:23 WARN TaskSetManager: Stage 5 contains a task of very large size (1940 KiB). The maximum recommended task size is 1000 KiB.
Root Mean Squared Error (RMSE) on test data: 0.09965964737349886


trainingData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: double, features: vector]
testData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: double, features: vector]
lr: org.apache.spark.ml.regression.LinearRegression = linReg_d01c5c5ec5f9
lrModel: org.apache.spark.ml.regression.LinearRegressionModel = LinearRegressionModel: uid=linReg_d01c5c5ec5f9, numFeatures=3
predictions: org.apache.spark.sql.DataFrame = [label: double, features: vector ... 1 more field]
evaluator: org.apache.spark.ml.evaluation.RegressionEvaluator = RegressionEvaluator: uid=regEval_c8e5eeab0991, metricName=rmse, throughOrigin=false
rmse: Double = 0.09965964737349886


# Batched LR

In [ ]:
val batchSize = 100
val coefficients = DenseVector(1.5, 0.3, -0.7)

val data = Seq.tabulate(131072) { _ =>
  val features = DenseVector.rand[Double](3)
  val label = coefficients.dot(features) + breeze.stats.distributions.Gaussian(0, 0.1).draw()
  (label, Vectors.dense(features.toArray))
}

val df = spark.createDataFrame(data).toDF("label", "features")
val numBatches = 131072 / batchSize
val batches = df.randomSplit(Array.fill(numBatches)(1.0))
var weights = DenseVector.zeros[Double](3 + 1)
for (batch <- batches) {
  val batchData = batch.toDF("label", "features")
  val labels = DenseVector(batchData.select("label").as[Double].collect())
  val features = batchData.select("features").rdd.map {
    case Row(v: Vector) => DenseVector(v.toArray :+ 1.0)
  }
  val featuresMatrix = DenseMatrix(features.collect():_*)
  val predictions = featuresMatrix * weights
  val errors = labels - predictions
  val gradient = -featuresMatrix.t * errors / batchSize.toDouble
  weights -= gradient
}
println(weights)

23/05/31 20:15:52 WARN TaskSetManager: Stage 7 contains a task of very large size (1940 KiB). The maximum recommended task size is 1000 KiB.
23/05/31 20:15:53 WARN TaskSetManager: Stage 8 contains a task of very large size (1940 KiB). The maximum recommended task size is 1000 KiB.
23/05/31 20:15:53 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/05/31 20:15:53 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
23/05/31 20:15:54 WARN TaskSetManager: Stage 9 contains a task of very large size (1940 KiB). The maximum recommended task size is 1000 KiB.
23/05/31 20:15:55 WARN TaskSetManager: Stage 10 contains a task of very large size (1940 KiB). The maximum recommended task size is 1000 KiB.
23/05/31 20:15:55 WARN TaskSetManager: Stage 11 contains a task of very large size (1940 KiB). The maximum recommended task size is 1000 KiB.
23/05/31 20:15:56 WARN TaskSetManager: Stage 12 contains a task of very large size